# SE446 – Week 2A Hands-On: Introduction to Big Data with Python

## 1. Introduction

In this notebook, we will **simulate key Big Data concepts** using a smaller scale environment (your laptop/Colab). Even though we aren't using a massive cluster yet, the *principles* remain the same.

**Learning Objectives:**
1. **Volume**: Observe how increasing data size affects memory and processing time.
2. **Variety**: Work with different data formats (CSV, Complex JSON, Unstructured Text).
3. **Veracity**: Identify dirty data and implement a basic cleaning pipeline.
4. **ETL vs ELT**: Understand the difference between transforming *before* loading vs *after* loading.
5. **File Formats**: Compare the storage and performance differences between **Row-based (CSV)** and **Columnar (Parquet)** formats.
6. **Pandas vs Spark**: Understand the difference between In-Memory and Distributed computing.

In [ ]:
# !pip install pandas
# !pip install pyarrow

In [3]:
# Setup - Import necessary libraries
import numpy as np
import pandas as pd
import json
import time
import sys
import os
from pathlib import Path

print(f"Pandas version: {pd.__version__}")

Pandas version: 3.0.0


---

## 2. Volume: The Challenge of Scale

### Objective
Big Data is often defined by **Volume**—datasets so large they cannot fit into the memory of a single machine. Here, we will simulate this by generating a "large" local dataset (1 million rows).

### What to do
1. Run the code to generate a synthetic dataset representing user activity.
2. Observe the **memory usage** reported by Pandas.
3. Observe the **time taken** to perform a simple aggregation query.

Try to imagine: *What would happen if this dataset were 1,000x larger?*

In [19]:
# ===============================================================
# 2.1 Generate Synthetic Dataset (1 Million Rows)
# ===============================================================

# N represents the number of simulated user interaction events
N = 1_000_000  # 1 million interactions

print(f"Generating {N:,} rows of data...")
np.random.seed(42)  # Ensures reproducible random results

# ---------------------------------------------------------------
# Create a DataFrame simulating user activity events
# ---------------------------------------------------------------

# Create a DataFrame simulating user purchase events
df = pd.DataFrame({
    "user_id": np.random.randint(1, 100_000, size=N),
    "event_type": np.random.choice(["click", "view", "purchase"], size=N, p=[0.6, 0.35, 0.05]),
    "amount": np.round(np.random.exponential(scale=50, size=N), 2),
    "timestamp": pd.date_range("2025-01-01", periods=N, freq="s")
})

print("Generation complete.")
df.head()

Generating 1,000,000 rows of data...
Generation complete.


,user_id,event_type,amount,timestamp
0,15796,view,44.07,2025-01-01 00:00:00
1,861,click,11.96,2025-01-01 00:00:01
2,76821,click,0.87,2025-01-01 00:00:02
3,54887,click,13.41,2025-01-01 00:00:03
4,6266,click,40.25,2025-01-01 00:00:04


In [20]:
# ===============================================================
# 2.2 Measure Memory Footprint and Query Time
# ===============================================================

print("--- Memory Usage ---")

# Display DataFrame info including RAM usage.
# memory_usage='deep' forces Pandas to compute the REAL memory size,
# including object overhead (not just raw array data).
df.info(memory_usage="deep")

print("\n--- Performance Test: Summing Revenue ---")

# Start timer to measure execution time of a simple aggregation
start = time.time()

# Filter only rows where the user actually purchased an item
# Then sum the 'amount' column to compute total revenue.
# This simulates a basic analytical query (GROUP BY-like operation).
total_revenue = df[df["event_type"] == "purchase"]["amount"].sum()

# Compute total query duration
duration = time.time() - start

# Display results in a readable format
print(f"Total revenue = ${total_revenue:,.2f}")
print(f"Query executed in: {duration:.4f} seconds")

# At this stage we learn two important Big Data concepts:
# 1. VOLUME → Even simple queries cost time when datasets grow.
# 2. MEMORY → DataFrames consume significant RAM, limiting scale on single machines.


--- Memory Usage ---
<class 'pandas.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   user_id     1000000 non-null  int64         
 1   event_type  1000000 non-null  str           
 2   amount      1000000 non-null  float64       
 3   timestamp   1000000 non-null  datetime64[us]
dtypes: datetime64[us](1), float64(1), int64(1), str(1)
memory usage: 35.1 MB

--- Performance Test: Summing Revenue ---
Total revenue = $2,486,083.23
Query executed in: 0.0120 seconds


### 💡 Concluding Remarks (Volume)
Even with just 1 million rows, the dataset takes up measureable RAM (approx 30MB+) and takes a fraction of a second to query. 

In a real Big Data scenario (e.g., Facebook or Google), datasets are **petabytes** in size. A single machine with 16GB RAM would crash instantly trying to load it. This motivates the need for **Distributed Computing** (Hadoop/Spark), where we split this data across 1,000 machines.

---

## 3. Variety: Handling Different Data Structures

### Objective
Data rarely comes in clean Excel spreadsheets. It often comes as **Semi-structured** logs (JSON) or **Unstructured** text or images.

### What to do
1. Read a traditional **CSV** (Structured).
2. Read a nested **JSON** file (Semi-structured) simulating IoT/Server logs.
3. Process simple **Text** lines (Unstructured).

In [21]:
# ===============================================================
# 3.1 Structured Data: CSV (Comma-Separated Values)
# ===============================================================
# CSV is an example of a **structured** file format.
# - It has a fixed schema (defined by column headers)
# - Data is stored in rows and columns
# - Easy to read with Pandas, Excel, SQL, etc.

csv_path = Path("transactions.csv")

# Save our DataFrame to a CSV file on disk
# 'index=False' avoids writing the DataFrame index as a column
df.to_csv(csv_path, index=False)

print(f"Saved CSV file: {csv_path}")

# Reading CSV back into a DataFrame is straightforward
# Pandas infers data types and loads it into memory
csv_df = pd.read_csv(csv_path)

# Show the first 2 rows as a quick validation check
csv_df.head(2)


Saved CSV file: transactions.csv


,user_id,event_type,amount,timestamp
0,15796,view,44.07,2025-01-01 00:00:00
1,861,click,11.96,2025-01-01 00:00:01


In [22]:
# ===============================================================
# 3.2 Semi-Structured Data: JSON (JavaScript Object Notation)
# ===============================================================
# JSON is a **semi-structured** format commonly used in APIs, logs, and NoSQL systems.
# Key characteristics:
# - Supports hierarchical (nested) fields
# - Keys can vary between records (flexible schema)
# - Not restricted to rows/columns like CSV

sample_events = [
    {"user_id": 1, "device": "mobile", "location": {"country": "SA", "city": "Riyadh"}},
    {"user_id": 2, "device": "web", "location": {"country": "SA", "city": "Jeddah"}},
    {"user_id": 3, "device": "tablet", "location": {"country": "AE", "city": "Dubai"}},
]

# ---------------------------------------------------------------
# Write JSON events to disk
# ---------------------------------------------------------------
# We write one JSON object per line (JSON Lines format), 
# which is commonly used for logs and streaming systems.
with open("events.json", "w") as f:
    for event in sample_events:
        f.write(json.dumps(event) + "\n")

# ---------------------------------------------------------------
# Read JSON with Schema-on-Read
# ---------------------------------------------------------------
# Pandas infers structure when reading the file.
# Notice how nested objects remain nested instead of flattened.

json_df = pd.read_json("events.json", lines=True)
print("Parsed Nested JSON:")
json_df

Parsed Nested JSON:


,user_id,device,location
0,1,mobile,"{'country': 'SA', 'city': 'Riyadh'}"
1,2,web,"{'country': 'SA', 'city': 'Jeddah'}"
2,3,tablet,"{'country': 'AE', 'city': 'Dubai'}"


**Why is `location` Unflattened? (Schema-on-Read Explanation)**

The `location` column remains unflattened because JSON is **semi-structured**, meaning nested fields and flexible keys are allowed. When we load the file with `pd.read_json(..., lines=True)`, Pandas applies **schema-on-read**: it infers structure at read time but does not automatically normalize nested objects into separate columns. Instead, it preserves the nested dictionary as a single cell so no information is lost. In real data systems (e.g., Spark, MongoDB, Data Lakes), schema-on-read allows applications to decide **later** how to interpret or flatten nested data depending on the query, making it flexible for evolving schemas and heterogeneous data sources.


In [ ]:
# ===============================================================
# 3.3 Unstructured Data: Free Text (Logs / Reviews / Messages)
# ===============================================================
# Unstructured data has **no predefined schema**:
# - No rows/columns
# - No fixed types
# - Meaning must be extracted manually (e.g., NLP, regex, ML)

raw_text = """
User 1: 'Service is slow but accurate'
User 2: 'Great price, fast delivery'
User 3: 'App keeps crashing, bad experience'
"""

# ---------------------------------------------------------------
# Convert raw multi-line text into a list of cleaned lines
# ---------------------------------------------------------------
# Steps:
# 1. Split by newline characters
# 2. Strip whitespace
# 3. Filter out empty lines
lines = [l for l in raw_text.split("\n") if l.strip()]

print(f"Processed {len(lines)} lines of text.")
print(lines[:2])  # Show first two lines as an example


Processed 3 lines of text.
["User 1: 'Service is slow but accurate'", "User 2: 'Great price, fast delivery'"]


### 💡 Concluding Remarks (Variety)
While CSVs are easy, modern Big Data systems deal heavily with JSON (APIs, MongoDB) and Unstructured data (reviews, images). Tools like **Spark** allow us to flatten nested JSON structures automatically, treating them like tables for analysis.

---

## 4. Veracity: The "Dirty Data" Reality

### Objective
Data in the real world is never clean. It has missing values (nulls), duplicates, and wrong formats. **Veracity** refers to the trustworthiness of data.

### What to do
1. Inject artificial errors into our dataset.
2. Perform a **Data Quality Check** to find the errors.
3. Execute a basic **Cleaning** step.

In [18]:
# ===============================================================
# 4.1 Create Dirty Data (Simulating Real-World Data Issues)
# ===============================================================

# Take a random sample of 10,000 rows from the clean dataset
# We use .copy() to avoid modifying the original DataFrame
dirty_df = df.sample(10_000).copy()

# ---------------------------------------------------------------
# Inject Artificial Data Quality Problems
# ---------------------------------------------------------------

# 1. Introduce missing values (NaN) in the 'amount' column
#    This simulates cases where payment amounts were not recorded.
dirty_df.loc[dirty_df.sample(500).index, "amount"] = np.nan

# 2. Introduce corrupted / unexpected category values in 'event_type'
#    This simulates data entry issues or unexpected API values.
dirty_df.loc[dirty_df.sample(300).index, "event_type"] = "UNKNOWN"

# ---------------------------------------------------------------
# Result Preview
# ---------------------------------------------------------------

print("Creating dirty dataset... done.")
dirty_df.head(10)

# At this point, dirty_df contains:
# - Missing numeric values
# - Invalid event categories
# This prepares us for the 'Veracity' step (data cleaning).


Creating dirty dataset... done.


,user_id,event_type,amount,timestamp
300266,35056,click,91.35,2025-01-04 11:24:26
308060,49570,click,42.09,2025-01-04 13:34:20
644152,66034,click,49.80,2025-01-08 10:55:52
657624,45226,view,17.63,2025-01-08 14:40:24
695424,32999,click,7.63,2025-01-09 01:10:24
706523,16619,purchase,5.20,2025-01-09 04:15:23
41681,39300,view,11.11,2025-01-01 11:34:41
459891,46132,UNKNOWN,61.79,2025-01-06 07:44:51
349608,69970,view,141.31,2025-01-05 01:06:48
5119,62523,click,33.10,2025-01-01 01:25:19


In [17]:
# ===============================================================
# 4.2 Quality Check & Cleaning
# ===============================================================

print("--- Data Quality Report ---")

# Count how many rows have missing revenue (amount = NaN)
print(f"Missing Amounts: {dirty_df['amount'].isna().sum()}")

# Count how many rows contain invalid/corrupted event values
print(f"Invalid Events: {dirty_df[dirty_df['event_type'] == 'UNKNOWN'].shape[0]}")

# ===============================================================
# CLEANING PIPELINE
# ===============================================================

# Always work on a copy to avoid changing original data unintentionally
clean_df = dirty_df.copy()

# ---------------------------------------------------------------
# Step 1: Imputation
# ---------------------------------------------------------------
# Replace missing amounts with 0 (simple strategy for demonstration)
# In real pipelines we might use mean/median imputation or ML models.
clean_df["amount"] = clean_df["amount"].fillna(0)

# ---------------------------------------------------------------
# Step 2: Filtering
# ---------------------------------------------------------------
# Keep only event values that we consider valid
# All rows with unexpected or corrupted event values are removed.
valid_events = ["click", "view", "purchase"]
clean_df = clean_df[clean_df["event_type"].isin(valid_events)]

# ---------------------------------------------------------------
# Summary of Cleaning
# ---------------------------------------------------------------
print("\n--- Cleaning Complete ---")
print(f"Original Row Count: {len(dirty_df)}")
print(f"Cleaned Row Count:  {len(clean_df)}")

# At this point:
# - All missing amounts have been replaced with 0
# - All invalid event types have been removed
# The cleaned dataset is now safe for analysis or ML.


--- Data Quality Report ---
Missing Amounts: 500
Invalid Events: 300

--- Cleaning Complete ---
Original Row Count: 10000
Cleaned Row Count:  9700


### 💡 Concluding Remarks (Veracity)
Before any analysis or Machine Learning can happen, data must be trusted. "Garbage In, Garbage Out" (GIGO) is the golden rule. Data Engineers spend a significant amount of time building these cleaning pipelines to ensure **Veracity**.

---

## 5. ETL vs ELT: Workflow Styles

### Objective
Understand the architectural difference between **ETL** (Extract, Transform, Load) and **ELT** (Extract, Load, Transform).

### What to do
1. **ETL Simulation**: We aggregate data *before* printing the final result. The raw details are discarded.
2. **ELT Simulation**: We "Load" (save) the raw data first. Then we transform it later on demand.

In [16]:
# user | amount | country
# A    | 100    | SA
# B    | 50     | SA
# A    | 70     | AE
# C    | 30     | SA

# ===============================================================
# Source Data (Raw)
# ===============================================================
# This represents transactional data BEFORE any processing.
# Each row is one transaction (user, amount, country).
raw_data = pd.DataFrame({
    "user": ["A", "B", "A", "C"],
    "amount": [100, 50, 70, 30],
    "country": ["SA", "SA", "AE", "SA"]
})

# ===============================================================
# 1. ETL Approach (Extract → Transform → Load)
# ===============================================================
# In ETL, we TRANSFORM the data BEFORE storing it.
# Here, aggregation summarizes total amount per country
# and only the SUMMARY results get saved/kept in the warehouse.
etl_aggregate = raw_data.groupby("country", as_index=False)["amount"].sum()
print("ETL Result (Stored in Warehouse):")
print(etl_aggregate)

# IMPORTANT:
# We lost row-level detail — for example:
# - User A had two separate transactions (100 and 70)
# - Country SA had three separate transactions (100, 50, 30)
# After aggregation, these multiple rows are merged into one,
# so individual transaction details cannot be recovered.
# That data CANNOT be recovered from the summary.
# This is what we mean by "loss of granularity" in ETL.


# ===============================================================
# 2. ELT Approach (Extract → Load → Transform)
# ===============================================================
# STEP 1: LOAD raw data "as-is" into storage (Data Lake).
# No transformation yet, so no data is lost.
raw_data.to_csv("raw_data_lake.csv", index=False)

# STEP 2: TRANSFORM later, on demand.
# Analysts or jobs can read the raw file and choose how to aggregate it.
lake_df = pd.read_csv("raw_data_lake.csv")
elt_aggregate = lake_df.groupby("country", as_index=False)["amount"].sum()

print("\nELT Result (Computed on demand):")
print(elt_aggregate)

# KEY DIFFERENCE:
# Raw data still exists, so we could ask new questions later
# (e.g., per-user stats, transaction counts), which is impossible with ETL.


ETL Result (Stored in Warehouse):
  country  amount
0      AE      70
1      SA     180

ELT Result (Computed on demand):
  country  amount
0      AE      70
1      SA     180


### 💡 Concluding Remarks (ETL vs ELT)
In **ETL**, transformations happen early. This saves storage but loses granularity.  
In **ELT** (Modern Data Lakes), we store *everything* first. This allows us to go back later and ask different questions (e.g., "What was User A's specific timestamp?") that would have been impossible with the ETL aggregate.

---

## 6. File Formats: CSV vs Parquet

### Objective
Big Data systems rarely use CSV for processing. They use binary columnar formats like **Parquet** or **ORC**. We will demonstrate why.

### What to do
1. Save our large dataset as both **CSV** and **Parquet**.
2. Compare the **File Size** (Compression).
3. Compare the **Read Speed** (Performance).

In [ ]:
# ===============================================================
# 6.1 Save Files in Different Formats (CSV vs Parquet)
# ===============================================================
# We extract only the columns needed for benchmarking storage formats.
# NOTE: Smaller subset = faster saves for demonstration purposes.
subset = df[["user_id", "event_type", "amount"]].copy()

# Define output file paths
csv_path = Path("events.csv")
parquet_path = Path("events.parquet")

# ---------------------------------------------------------------
# Save as CSV (Row-Based Format)
# ---------------------------------------------------------------
# CSV writes plain text, row by row.
# This format does NOT apply compression or type optimization.
print("Saving CSV... (this might take a moment)")
subset.to_csv(csv_path, index=False)

# ---------------------------------------------------------------
# Save as Parquet (Columnar Format)
# ---------------------------------------------------------------
# Parquet is a binary, columnar format optimized for analytics.
# It applies compression under the hood (e.g., Snappy).
print("Saving Parquet...")
subset.to_parquet(parquet_path, index=False)

# ---------------------------------------------------------------
# Compare File Sizes
# ---------------------------------------------------------------
# Convert bytes → megabytes for human readability.
csv_size = csv_path.stat().st_size / (1024 * 1024)
pq_size = parquet_path.stat().st_size / (1024 * 1024)

print(f"\nCSV Size:     {csv_size:.2f} MB")
print(f"Parquet Size: {pq_size:.2f} MB")
print(f"Compression:  Parquet is {csv_size/pq_size:.1f}x smaller!")

# This demonstrates:
# - CSV = simple, readable, but storage-heavy
# - Parquet = compact, binary, columnar, analytics-friendly
# Parquet's efficiency becomes critical at Big Data scale.


Saving CSV... (this might take a moment)
Saving Parquet...

CSV Size:     16.73 MB
Parquet Size: 4.61 MB
Compression:  Parquet is 3.6x smaller!


In [ ]:
# ===============================================================
# 6.2 Benchmark Read Speed (CSV vs Parquet)
# ===============================================================
# We measure how long it takes to:
# 1. Read the file from disk
# 2. Select the `amount` column
# 3. Compute the mean
#
# The %timeit magic command runs multiple repetitions to get
# a stable average runtime, reducing noise caused by the system.

print("\n--- Benchmarking Read Speed (Average of 3 runs) ---")

# ---------------------------------------------------------------
# Benchmark: CSV (Row-Based Format)
# ---------------------------------------------------------------
# CSV must parse the file line-by-line, converting text → numbers.
# This is slower because we read ALL columns and ALL rows as text.
print("Reading CSV (Row-based)...")
%timeit -n3 -r3 pd.read_csv(csv_path)["amount"].mean()

# ---------------------------------------------------------------
# Benchmark: Parquet (Columnar Format)
# ---------------------------------------------------------------
# Parquet stores data in binary, by columns, with schema included.
# For analytical queries (like computing mean of one column),
# Parquet can read just the needed column, making it more efficient.
print("\nReading Parquet (Columnar-based)...")
%timeit -n3 -r3 pd.read_parquet(parquet_path)["amount"].mean()

# NOTE:
# For small files and local machines, differences may vary
# because Parquet has some overhead (metadata + decompression).
# At large scale (GB–TB), columnar formats are much faster.



--- Benchmarking Read Speed (Average of 3 runs) ---
Reading CSV (Row-based)...
128 ms ± 4.54 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)

Reading Parquet (Columnar-based)...
The slowest run took 58.69 times longer than the fastest. This could mean that an intermediate result is being cached.
298 ms ± 401 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


### 💡 Concluding Remarks (Formats)
**Parquet is significantly faster and smaller.**

*   **Size**: Parquet uses **smart compression** (like run-length encoding), making it much cheaper to store.
*   **Speed**: Because it is **Columnar**, calculating the average `amount` only requires reading that one column. The CSV reader must parse every single row text-by-line, which is slow.

Use **Columnar formats** (Parquet/ORC) for Big Data analytics!

Here’s a clear **~150-word educational explanation** of the results:

---
### Concluding Remarks (Formats)
This experiment compares CSV and Parquet as data storage formats. When we saved the same subset of data, the CSV file took about **16.7 MB**, while the Parquet file took only **4.6 MB**. This means Parquet was roughly **3.6× smaller**. The reason is that Parquet uses **columnar storage** and **built-in compression**, while CSV stores everything as plain text with no compression.

Then we benchmarked read performance. Reading the CSV took around **128 ms**, while Parquet showed a slower and more variable time (around **298 ms ± large variation**). This may seem surprising because Parquet is often faster on big analytics workloads, but here we only read a small file on a single machine. In this situation, the overhead of decompression and metadata handling can outweigh the benefits.

So the takeaway is: **CSV is simple and good for small data**, while **Parquet is space-efficient and optimized for large-scale analytics, Spark, and columnar queries**.


---

## 7. Pandas vs Spark DataFrames: Why Big Data Needs Distributed DataFrames

### Objective
**Learning Objectives:**
1. **Execution Model**: Understand *where* your code runs (Single CPU vs Cluster).
2. **Overhead vs. Scale**: See why Spark is slower for small data but necessary for big data.
3. **Lazy Evaluation**: Learn that Spark (unlike Pandas) doesn't compute until you ask for results (like `.show()`).

### Understanding the Complexity: Why is Spark Harder to Run?
You might wonder: *"Why do we need Java? Why all these environment variables? Why do we need to restart the runtime?"*

Great question! Spark is complex to run because of its **distributed computing architecture**. Here's the simple explanation:

#### 1. Spark vs Regular Python

**Regular Python (Pandas)**
```
┌─────────────────────┐
│   Your Computer     │
│   Python → CPU      │
└─────────────────────┘
      Simple!
```

**Spark**
```
┌─────────────────────────────────────────────┐
│  Your Code (Python)                         │
│       ↓                                     │
│  PySpark (Python library)                   │
│       ↓                                     │
│  Py4J (Python-to-Java bridge)               │
│       ↓                                     │
│  Spark Core (Java/Scala - needs JVM)        │
│       ↓                                     │
│  Cluster Manager (even "local" mode)        │
│       ↓                                     │
│  Workers (parallel processing)              │
└─────────────────────────────────────────────┘
      Many layers!
```

#### 2. Why So Many Dependencies?

| Dependency | Why It's Needed |
|------------|-----------------|
| **Java (JVM)** | Spark is written in Scala, which runs on Java |
| **Py4J** | Translates Python calls to Java |
| **Hadoop libraries** | File system handling (HDFS, S3, etc.) |
| **Environment variables** | So all pieces can find each other |

#### 3. The Tradeoff

| | Pandas | Spark |
|--|--------|-------|
| **Setup** | `pip install pandas` ✅ | Java + PySpark + config 😅 |
| **Data size** | ~1-10 GB (RAM limit) | Petabytes across clusters |
| **Speed (small data)** | **Faster** | Slower (overhead) |
| **Speed (big data)** | Crashes 💥 | **Handles it easily** ✅ |

#### ⚠️ IMPORTANT: Run the Installation Cell first, then RESTART RUNTIME!
Colab requires a runtime restart after changing environment variables (Java). Go to **Runtime -> Restart Session** after running Step 1.

In [ ]:
# ===============================================================
# STEP 1: Initialize Spark Session
# ===============================================================
# IMPORTANT: In Colab, you must restart the runtime after installing Java + PySpark
# before running this cell, otherwise Spark will fail to start.

import os
import sys
import platform

# ---------------------------------------------------------------
# Configure JAVA_HOME (Spark runs on the JVM)
# ---------------------------------------------------------------
# We detect the environment and set the correct Java path.
# - Google Colab uses system OpenJDK
# - macOS uses Homebrew path (if installed that way)
# - Linux and Windows users may need manual adjustments

if 'google.colab' in sys.modules:
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
else:
    if platform.system() == "Darwin":  # macOS
        os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11/libexec/openjdk.jdk/Contents/Home"
    elif platform.system() == "Linux":  # Linux Desktop
        os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
    # Windows users: set JAVA_HOME manually if needed

# Tell PySpark to use the current Python interpreter
os.environ["PYSPARK_PYTHON"] = sys.executable

from pyspark.sql import SparkSession

# ---------------------------------------------------------------
# Create Spark Session
# ---------------------------------------------------------------
# .master("local[*]") means:
# - Run Spark locally
# - Use as many CPU cores as available
#
# Even though this is "local mode", Spark still uses the full
# distributed execution engine architecture under the hood.
spark = SparkSession.builder \
    .appName("BigDataDemo") \
    .master("local[*]") \
    .getOrCreate()

print(f"✅ Spark Version: {spark.version}")
print("✅ Spark Session Created!")

# ---------------------------------------------------------------
# Quick Test DataFrame
# ---------------------------------------------------------------
# If the following prints a small table, Spark is working correctly.
df = spark.createDataFrame([(1, "hello"), (2, "spark")], ["id", "value"])
df.show()

# At this point:
# - Spark is running on your machine
# - PySpark is bridging Python ↔ JVM
# - Ready for distributed DataFrame operations


26/01/25 21:55:22 WARN Utils: Your hostname, Anis-Koubaas-MacBook-Pro-10.local resolves to a loopback address: 127.0.0.1; using 192.168.1.197 instead (on interface en0)
26/01/25 21:55:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/25 21:55:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark Version: 3.5.0
✅ Spark Session Created!


+---+-----+
| id|value|
+---+-----+
|  1|hello|
|  2|spark|
+---+-----+



In [ ]:
# ===============================================================
# STEP 2: Initialize Spark Session
# ===============================================================
# NOTE: In Colab, Spark requires Java. After installing Java/JDK
# and setting environment variables in the previous step,
# you MUST restart the runtime before running this cell.

import os
import sys

# ---------------------------------------------------------------
# Re-set environment variables after restart
# ---------------------------------------------------------------
# JAVA_HOME tells PySpark where to find the JVM (Java Virtual Machine)
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Add Java binaries to PATH so the 'java' command is available
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

import pyspark
from pyspark.sql import SparkSession

# ---------------------------------------------------------------
# Create SparkSession (entry point to Spark functionality)
# ---------------------------------------------------------------
# Explanation:
# - .appName("BigDataDemo") names the application
# - .master("local[*]") runs Spark in local mode
#      using all available CPU cores
# - .config("spark.driver.memory", "4g") allocates memory to driver
#
# Even in "local" mode, Spark uses its distributed execution engine.
try:
    spark = SparkSession.builder \
        .appName("BigDataDemo") \
        .master("local[*]") \
        .config("spark.driver.memory", "4g") \
        .getOrCreate()
    
    print(f"✅ Spark Version: {spark.version}")
    print("✅ Spark Session Created!")

# ---------------------------------------------------------------
# Common error handling
# ---------------------------------------------------------------
# If Java is not configured correctly, PySpark may throw:
# 'JavaPackage object is not callable'
# This indicates that JAVA_HOME wasn't recognized at startup.
except TypeError as e:
    print("\n❌ ERROR: Spark could not connect to Java.")
    print("If you see 'JavaPackage object is not callable', restart the runtime, then re-run this cell.")
    print("Menu: Runtime → Restart Session")
    raise e

# ---------------------------------------------------------------
# Quick sanity check: create a tiny DataFrame
# ---------------------------------------------------------------
# If this prints as a table, Spark is working correctly.
df_check = spark.createDataFrame([(1, "test"), (2, "spark")], ["id", "value"])
df_check.show()

# At this point Spark is ready for:
# - distributed DataFrame operations
# - reading Parquet/JSON/CSV in distributed mode
# - running SQL queries via spark.sql()


✅ Spark Version: 3.5.0
✅ Spark Session Created!


26/01/25 21:55:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---+-----+
| id|value|
+---+-----+
|  1| test|
|  2|spark|
+---+-----+



# Step 3: Create DataFrames (Pandas vs Spark)

### Creating DataFrames: The "Hello World" of Spark

Now that Spark is running, let's create some data to compare.
1.  **Pandas**: We create a DataFrame in **local memory**. Easy and fast for this size.
2.  **Spark**: We convert that Pandas DF into a **Distributed DataFrame**. 
    *   *Note*: In a real scenario, you would read from S3/HDFS directly. Converting local Pandas -> Spark varies in speed but is good for demos.
    
This simulates "Ingestion".

## What's Happening Here?

1. **Create data with Pandas** → Lives in your computer's RAM (single machine)
2. **Convert to Spark DataFrame** → Data is now distributed and parallelized

## Key Difference

| Pandas DataFrame | Spark DataFrame |
|------------------|-----------------|
| Stored in RAM | Distributed across workers |
| Processed by 1 CPU | Processed by many CPUs in parallel |
| `pdf` (Python object) | `sdf` (pointer to distributed data) |

## Why Convert?

Pandas is great for creating/loading data, but Spark can **process it in parallel** — essential when data grows to billions of rows.


In [ ]:
# ===============================================================
# STEP 3: Create DataFrames (Pandas vs Spark)
# ===============================================================
import pandas as pd
import numpy as np

# ---------------------------------------------------------------
# A. Generate Data in Pandas (In-Memory)
# ---------------------------------------------------------------
# We create 1,000,000 rows directly in RAM using Pandas.
# Pandas DataFrames live on a single machine and use a single CPU.
N = 1_000_000
print(f"Generating {N:,} rows in Pandas...")

pdf = pd.DataFrame({
    "user_id": np.random.randint(1, 100_000, size=N),  # Many users, repeated IDs
    "amount": np.round(np.random.exponential(scale=50, size=N), 2),  # Skewed monetary values
    "event": np.random.choice(["click", "view", "purchase"], size=N)  # Categorical events
})

# ---------------------------------------------------------------
# B. Convert Pandas DataFrame to Spark DataFrame (Distributed)
# ---------------------------------------------------------------
# Converting to a Spark DataFrame allows distributed operations.
# In a real use case, Spark would read from files (S3/HDFS/Parquet).
print("Converting to Spark DataFrame...")

sdf = spark.createDataFrame(pdf)

print("Done.")

# Display the first 5 rows of the Spark DataFrame
# This triggers a Spark action (lazy evaluation until .show


Generating 1,000,000 rows in Pandas...
Converting to Spark DataFrame...
Done.
+-------+------+--------+
|user_id|amount|   event|
+-------+------+--------+
|  95042| 84.94|   click|
|  97917| 56.52|purchase|
|  50646| 22.82|    view|
|  86286| 20.84|   click|
|  66212| 55.16|purchase|
+-------+------+--------+
only showing top 5 rows



26/01/25 21:55:39 WARN TaskSetManager: Stage 6 contains a task of very large size (1902 KiB). The maximum recommended task size is 1000 KiB.


# **Phase 3 Output — Key Takeaways**

## **About the Warnings**

You may see warnings such as:

| Warning                                  | Meaning                                                                 |
|------------------------------------------|-------------------------------------------------------------------------|
| `Task of very large size (1901 KiB)`     | Spark prefers smaller partitions; we sent ~1M rows as one block.        |
| `Detected deadlock`                      | Temporary stall between Python ↔ JVM; Spark recovers automatically.     |

> **Note:** These warnings are common when running Spark locally with small datasets.
> Spark is optimized for distributed clusters and massive data workloads.

---

# **Performance Difference (Pandas vs Spark)**

When benchmarking with `%time`:

### **What `%time` Reports**
| Metric        | Meaning                               |
|---------------|---------------------------------------|
| **CPU time**  | Actual computation time                |
| **Wall time** | Total real-world waiting time          |

**Example:** CPU = 5 ms vs Wall = 600 ms → most time spent waiting (setup/communication).

---

### **Why Pandas Is Faster Here**

**Pandas**
- Data is already in your computer’s memory (RAM)
- Runs directly on the CPU
- No extra setup or communication steps  
➡️ **So it's fast (usually milliseconds)**

**Spark**
1. Needs to create tasks
2. Needs to organize workers (even if only 1 worker on your laptop)
3. Needs to send data between **Python** and **Java (JVM)**
4. Waits until an action is called (lazy evaluation)  
➡️ **So there is a startup cost (usually seconds)**

👉 **Important:** Spark is not slower because it's bad — Spark is designed for **huge datasets and clusters (+10M)**, not tiny demos on a laptop. For big data, Spark wins. For small data, Pandas wins.


---

## **Core Lesson**
Spark has a **fixed startup cost**. For **small data**, overhead dominates and Spark feels slow.

For **large data (GB–TB)**, Spark’s parallelism becomes a major advantage.



In [ ]:
# ===============================================================
# STEP 4: Compare Computations (Pandas vs Spark)
# ===============================================================

print("--- Pandas (Single-Core) ---")
# ---------------------------------------------------------------
# Pandas executes eagerly and in-memory on a single CPU core.
# %time measures how long the groupby + mean operation takes.
# ---------------------------------------------------------------
%time pdf.groupby("event")["amount"].mean()

print("\n--- Spark (Distributed Plan) ---")
# ---------------------------------------------------------------
# Spark uses lazy evaluation: it builds a logical execution plan
# but does not actually run it until an ACTION is triggered.
#
# groupBy(...).avg(...) alone does nothing until .show(), .collect(),
# .count(), or similar actions are called.
#
# %time measures the time to:
#   1. plan the distributed job
#   2. schedule tasks
#   3. execute the aggregation
#   4. collect results to the driver for printing
# ---------------------------------------------------------------
%time sdf.groupBy("event").avg("amount").show()  # .show() triggers execution


--- Pandas (Single-Core) ---
CPU times: user 45 ms, sys: 25 ms, total: 70 ms
Wall time: 82.4 ms

--- Spark (Distributed Plan) ---


26/01/25 21:58:58 WARN TaskSetManager: Stage 10 contains a task of very large size (1902 KiB). The maximum recommended task size is 1000 KiB.


+--------+------------------+
|   event|       avg(amount)|
+--------+------------------+
|purchase| 50.01232078702823|
|    view|49.962862479696966|
|   click|50.058101695830004|
+--------+------------------+

CPU times: user 2.57 ms, sys: 2.24 ms, total: 4.8 ms
Wall time: 769 ms


Pandas completes the aggregation in ~80 ms because it runs eagerly in-memory on one core with no coordination overhead. Spark takes ~770 ms because it must plan, schedule, and coordinate a distributed job before executing. For small data, Spark’s startup overhead dominates; for large data, its parallelism becomes beneficial.

### 💡 Concluding Remarks (Pandas vs Spark)

*   **Pandas** is incredibly fast for data that fits in RAM (like this 1M row example), often beating Spark due to low overhead.
*   **Spark** has overhead (starting tasks, communicating), so it might seem slower here.
*   **HOWEVER**: If we had **1 Billion rows**, Pandas would crash with an `Out of Memory` error. Spark would simply split the data into chunks and process it in parallel, taking longer but completing successfully.

**Rule of Thumb:** Use Pandas for MBs/GBs. Use Spark for TBs/PBs.

---

## 8. Short Reflection

**Instructions:**
Based on the exercises above, write a short reflection (3-5 sentences) answering the following:

1. Which activity demonstrated the biggest performance difference?
2. Why do you think "Schema-on-Read" (handling JSON) is important for modern apps versus traditional SQL tables?
3. Why would a company prefer ELT (Data Lake) over ETL (Data Warehouse) today?
4. Why does Spark feel slower than Pandas for small data, but is preferred for Big Data?

*(Double-click here to write your answer)*